In [282]:
# Setup: Add project root to path and import utilities
import sys
sys.path.insert(0, '..')

from utils import get_input
import numpy as np
from math import prod

# Load input data for day 9
data = get_input(9, example=False)

print(f"Loaded {len(data)} lines of input")
print("Sample data:")
for line in data[:5]:
    print(f"  {line}")

Loaded 496 lines of input
Sample data:
  97926,50400
  97926,51618
  98025,51618
  98025,52819
  97711,52819


In [283]:
def show_grid(grid_data, sample_size=50):
    """
    Display a grid overview with coordinates marked as '#'.
    For large grids, shows a sample of points instead of full grid.
    """
    if not grid_data:
        return
    
    xs = [coord[0] for coord in grid_data]
    ys = [coord[1] for coord in grid_data]
    
    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)
    
    width = max_x - min_x + 1
    height = max_y - min_y + 1
    
    print(f"Grid bounds: ({min_x}, {min_y}) to ({max_x}, {max_y})")
    print(f"Grid size: {width} x {height}")
    print(f"Total points: {len(grid_data)}")
    print(f"\nFirst {min(sample_size, len(grid_data))} points:")
    
    for i, (x, y) in enumerate(grid_data[:sample_size]):
        print(f"  {i+1}. ({x}, {y})")
        if i >= sample_size - 1:
            break
    
    if len(grid_data) > sample_size:
        print(f"  ... and {len(grid_data) - sample_size} more points")

In [284]:
# Parse input data into list of coordinate tuples
grid_data = [tuple(int(coord) for coord in line.split(',')) for line in data]

In [285]:
show_grid(grid_data)

Grid bounds: (1515, 1666) to (98234, 98094)
Grid size: 96720 x 96429
Total points: 496

First 50 points:
  1. (97926, 50400)
  2. (97926, 51618)
  3. (98025, 51618)
  4. (98025, 52819)
  5. (97711, 52819)
  6. (97711, 54079)
  7. (98234, 54079)
  8. (98234, 55220)
  9. (97394, 55220)
  10. (97394, 56444)
  11. (97431, 56444)
  12. (97431, 57586)
  13. (96897, 57586)
  14. (96897, 58805)
  15. (96869, 58805)
  16. (96869, 59983)
  17. (96599, 59983)
  18. (96599, 61256)
  19. (96728, 61256)
  20. (96728, 62403)
  21. (96305, 62403)
  22. (96305, 63526)
  23. (95822, 63526)
  24. (95822, 64731)
  25. (95613, 64731)
  26. (95613, 65925)
  27. (95349, 65925)
  28. (95349, 67175)
  29. (95211, 67175)
  30. (95211, 68325)
  31. (94784, 68325)
  32. (94784, 69209)
  33. (93751, 69209)
  34. (93751, 70557)
  35. (93772, 70557)
  36. (93772, 71327)
  37. (92575, 71327)
  38. (92575, 72350)
  39. (91929, 72350)
  40. (91929, 73494)
  41. (91506, 73494)
  42. (91506, 74481)
  43. (90803, 74481)
 

In [286]:
def calculate_rectangle_area(x1, y1, x2, y2):
    """Calculate the area of a rectangle defined by two corner points."""
    return abs((x2 - x1 + 1) * (y2 - y1 + 1))


def find_largest_rectangle_area(coordinates):
    """Find the largest rectangle area formed by any two points in the coordinate list."""
    if len(coordinates) < 2:
        return 0
    
    largest_area = 0
    for i, (x1, y1) in enumerate(coordinates):
        for x2, y2 in coordinates[i + 1:]:
            area = calculate_rectangle_area(x1, y1, x2, y2)
            largest_area = max(largest_area, area)
    
    return largest_area


# Find largest rectangle
largest_area = find_largest_rectangle_area(grid_data)
print(f"Largest rectangle area: {largest_area}")

Largest rectangle area: 4748493456


In [287]:
class Grid:
    """Represents a grid with red tiles (boundary) and green tiles (interior)."""
    
    def __init__(self, boundary_points, compute_full_interior=False):
        """
        Initialize grid with boundary points.
        
        Args:
            boundary_points: Ordered list of (x, y) tuples forming polygon boundary
            compute_full_interior: If True, pre-compute all interior tiles (slow for large grids)
        """
        self.red_tiles = boundary_points
        self.red_tiles_set = set(boundary_points)
        self.edge_tiles = self._create_edge_tiles()
        self._inside_cache = {}  # Cache for interior point checks
        
        if compute_full_interior:
            self.green_tiles = self._create_all_interior_tiles()
        else:
            self.green_tiles = list(self.edge_tiles)
    
    def _get_bounds(self):
        """Get the bounding box of the grid."""
        if not self.red_tiles:
            return 0, 0, 0, 0
        
        xs = [x for x, _ in self.red_tiles]
        ys = [y for _, y in self.red_tiles]
        return min(xs), min(ys), max(xs), max(ys)
    
    def _create_edge_tiles(self):
        """Create tiles along edges between consecutive boundary points (only on straight lines)."""
        edge_tiles = set()
        
        for i in range(len(self.red_tiles)):
            x1, y1 = self.red_tiles[i]
            x2, y2 = self.red_tiles[(i + 1) % len(self.red_tiles)]
            
            # Tiles are on same row or same column (straight line)
            if x1 == x2:  # Vertical line
                for y in range(min(y1, y2), max(y1, y2) + 1):
                    if (x1, y) not in self.red_tiles_set:
                        edge_tiles.add((x1, y))
            elif y1 == y2:  # Horizontal line
                for x in range(min(x1, x2), max(x1, x2) + 1):
                    if (x, y1) not in self.red_tiles_set:
                        edge_tiles.add((x, y1))
        
        return edge_tiles
    
    def _is_point_inside_polygon(self, x, y):
        """
        Check if point (x, y) is inside polygon using ray-casting algorithm.
        
        Args:
            x, y: Coordinates to test
            
        Returns:
            bool: True if point is inside polygon
        """
        if (x, y) in self._inside_cache:
            return self._inside_cache[(x, y)]
        
        inside = False
        n = len(self.red_tiles)
        j = n - 1
        
        for i in range(n):
            xi, yi = self.red_tiles[i]
            xj, yj = self.red_tiles[j]
            
            # Check if ray from point crosses edge
            if ((yi > y) != (yj > y)) and (x < (xj - xi) * (y - yi) / (yj - yi) + xi):
                inside = not inside
            j = i
        
        self._inside_cache[(x, y)] = inside
        return inside
    
    def is_tile_valid(self, x, y):
        """Check if a tile at (x, y) is red or green (valid for rectangles)."""
        if (x, y) in self.red_tiles_set or (x, y) in self.edge_tiles:
            return True
        return self._is_point_inside_polygon(x, y)
    
    def _create_all_interior_tiles(self):
        """
        Create all interior tiles (green tiles) including edges and polygon interior.
        Only call this for small grids!
        
        Returns:
            list: List of (x, y) tuples representing interior tiles
        """
        min_x, min_y, max_x, max_y = self._get_bounds()
        
        interior_tiles = set(self.edge_tiles)
        
        # Find all points inside the polygon
        print(f"Computing all interior tiles...")
        for x in range(min_x, max_x + 1):
            for y in range(min_y, max_y + 1):
                if (x, y) not in self.red_tiles_set and (x, y) not in interior_tiles:
                    if self._is_point_inside_polygon(x, y):
                        interior_tiles.add((x, y))
        
        return list(interior_tiles)
    
    def find_largest_rectangle(self):
        """
        Find the largest rectangle that fits within the grid (red + green tiles).
        Uses optimized checking with early exit and size limits.
        
        Returns:
            tuple: (area, (x1, y1, x2, y2)) or (0, None) if no valid rectangle
        """
        largest_area = 0
        best_rectangle = None
        
        # Sort red tiles to try more promising pairs first
        red_list = list(self.red_tiles)
        
        print(f"Checking rectangles among {len(red_list)} red tiles...")
        checked = 0
        max_rect_size = 1000000  # Don't check rectangles larger than this
        
        for i, (x1, y1) in enumerate(red_list):
            if i % 50 == 0 and i > 0:
                print(f"  Progress: {i}/{len(red_list)}, best so far: {largest_area}, checked: {checked}")
            
            for j, (x2, y2) in enumerate(red_list):
                if i >= j:  # Skip same point and duplicates
                    continue
                
                # Calculate potential area
                rect_area = calculate_rectangle_area(x1, y1, x2, y2)
                
                # Skip if can't be better
                if rect_area <= largest_area:
                    continue
                
                # Skip if rectangle is too large to check efficiently
                if rect_area > max_rect_size:
                    continue
                
                # Check if all tiles in rectangle are valid
                checked += 1
                if self._is_rectangle_valid_fast(x1, y1, x2, y2):
                    largest_area = rect_area
                    best_rectangle = (x1, y1, x2, y2)
        
        print(f"  Total rectangles checked: {checked}")
        return largest_area, best_rectangle
    
    def _is_rectangle_valid_fast(self, x1, y1, x2, y2):
        """Check if all tiles in rectangle are red or green with early exit."""
        min_x, max_x = min(x1, x2), max(x1, x2)
        min_y, max_y = min(y1, y2), max(y1, y2)
        
        # Check perimeter first (most likely to fail)
        for x in range(min_x, max_x + 1):
            if not self.is_tile_valid(x, min_y):
                return False
            if not self.is_tile_valid(x, max_y):
                return False
        
        for y in range(min_y + 1, max_y):
            if not self.is_tile_valid(min_x, y):
                return False
            if not self.is_tile_valid(max_x, y):
                return False
        
        # Check interior
        for x in range(min_x + 1, max_x):
            for y in range(min_y + 1, max_y):
                if not self.is_tile_valid(x, y):
                    return False
        
        return True

In [288]:
def display_grid(grid: Grid, sample_size=50):
    """
    Display grid overview with red and green tiles.
    For large grids, shows statistics instead of visual representation.
    """
    if not grid.red_tiles:
        return
    
    all_points = grid.red_tiles + grid.green_tiles
    xs = [coord[0] for coord in all_points]
    ys = [coord[1] for coord in all_points]
    
    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)
    
    width = max_x - min_x + 1
    height = max_y - min_y + 1
    
    print(f"Grid: {width} x {height}")
    print(f"Red tiles (boundary): {len(grid.red_tiles)}")
    print(f"Green tiles (interior): {len(grid.green_tiles)}")
    print(f"Total coverage: {len(all_points)} tiles")
    print(f"\nSample of first {min(sample_size, len(grid.red_tiles))} red tiles:")
    
    for i, (x, y) in enumerate(grid.red_tiles[:sample_size]):
        print(f"  {i+1}. ({x}, {y})")
        if i >= sample_size - 1:
            break


def display_grid_with_rectangle(grid: Grid, rectangle, sample_size=50):
    """
    Display grid with rectangle information.
    For large grids, shows statistics instead of visual representation.
    """
    if not rectangle or not grid.red_tiles:
        return
    
    x1, y1, x2, y2 = rectangle
    all_points = grid.red_tiles + grid.green_tiles
    xs = [coord[0] for coord in all_points]
    ys = [coord[1] for coord in all_points]
    
    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)
    
    width = max_x - min_x + 1
    height = max_y - min_y + 1
    
    rect_width = abs(x2 - x1) + 1
    rect_height = abs(y2 - y1) + 1
    
    print(f"Grid: {width} x {height}")
    print(f"Red tiles: {len(grid.red_tiles)}, Green tiles: {len(grid.green_tiles)}")
    print()
    print(f"Rectangle corners: ({x1}, {y1}) to ({x2}, {y2})")
    print(f"Rectangle size: {rect_width} x {rect_height}")
    print(f"Coverage: {(rect_width * rect_height / (width * height) * 100):.2f}% of grid area")

In [289]:
# Create grid and display tiles
grid = Grid(grid_data)
print(f"Number of red tiles (boundary): {len(grid.red_tiles)}")
print(f"Number of green tiles (interior): {len(grid.green_tiles)}")
print()

# Uncomment to display the grid
# display_grid(grid)

Number of red tiles (boundary): 496
Number of green tiles (interior): 586138



In [290]:
# Find and display the largest rectangle
area, rectangle = grid.find_largest_rectangle()

if rectangle:
    print(f"Largest rectangle area: {area}")
    print(f"Rectangle corners: {rectangle}")
    print()
    display_grid_with_rectangle(grid, rectangle)
else:
    print("No valid rectangle found")

Checking rectangles among 496 red tiles...


KeyboardInterrupt: 

In [ ]:
# Debug: Let's check what the actual problem is
print("Checking the grid structure:")
print(f"Red tiles: {len(grid.red_tiles)}")
print(f"Green tiles (edges): {len(grid.green_tiles)}")
print(f"All tiles: {len(grid.red_tiles) + len(grid.green_tiles)}")
print()

# Check if consecutive points form edges
sample_edges = []
for i in range(min(5, len(grid.red_tiles))):
    x1, y1 = grid.red_tiles[i]
    x2, y2 = grid.red_tiles[(i + 1) % len(grid.red_tiles)]
    distance = abs(x2 - x1) + abs(y2 - y1)
    sample_edges.append(((x1, y1), (x2, y2), distance))
    print(f"Edge {i}: ({x1},{y1}) -> ({x2},{y2}), Manhattan distance: {distance}")

print()
print("The issue: consecutive boundary points are FAR apart!")
print("This creates huge rectangles that are mostly empty space.")

Checking the grid structure:
Red tiles: 496
Green tiles (edges): 586138
All tiles: 586634

Edge 0: (97926,50400) -> (97926,51618), Manhattan distance: 1218
Edge 1: (97926,51618) -> (98025,51618), Manhattan distance: 99
Edge 2: (98025,51618) -> (98025,52819), Manhattan distance: 1201
Edge 3: (98025,52819) -> (97711,52819), Manhattan distance: 314
Edge 4: (97711,52819) -> (97711,54079), Manhattan distance: 1260

The issue: consecutive boundary points are FAR apart!
This creates huge rectangles that are mostly empty space.


In [ ]:
# Alternative interpretation: Find largest rectangle bounded by any two red tiles
# without requiring all interior tiles to exist
def find_largest_rectangle_simple(boundary_points):
    """Find largest rectangle using any two boundary points as corners."""
    largest_area = 0
    best_rectangle = None
    
    for i, (x1, y1) in enumerate(boundary_points):
        for x2, y2 in boundary_points[i + 1:]:
            if x1 != x2 and y1 != y2:  # Skip degenerate cases (lines)
                area = calculate_rectangle_area(x1, y1, x2, y2)
                if area > largest_area:
                    largest_area = area
                    best_rectangle = (x1, y1, x2, y2)
    
    return largest_area, best_rectangle

# Test this interpretation
simple_area, simple_rect = find_largest_rectangle_simple(grid.red_tiles)
print(f"Largest rectangle (simple, any two corners):")
print(f"Area: {simple_area}")
print(f"Corners: {simple_rect}")
if simple_rect:
    x1, y1, x2, y2 = simple_rect
    print(f"Dimensions: {abs(x2-x1)+1} x {abs(y2-y1)+1}")

Largest rectangle (simple, any two corners):
Area: 4748493456
Corners: (86377, 82538, 16500, 14581)
Dimensions: 69878 x 67958


In [ ]:
# Test with example data to verify logic
data_example = get_input(9, example=True)
grid_data_example = [tuple(int(coord) for coord in line.split(',')) for line in data_example]

print(f"Example data: {len(grid_data_example)} points")
grid_example = Grid(grid_data_example)
print(f"\nRed tiles: {len(grid_example.red_tiles)}")
print(f"Green tiles: {len(grid_example.green_tiles)}")
print(f"Total: {len(grid_example.red_tiles) + len(grid_example.green_tiles)}")

Example data: 8 points

Red tiles: 8
Green tiles: 22
Total: 30


In [ ]:
# Find largest rectangle in example
area_ex, rect_ex = grid_example.find_largest_rectangle()
print(f"\nLargest rectangle in example:")
print(f"Area: {area_ex}")
print(f"Corners: {rect_ex}")

# Visualize example (small enough to display)
print("\nExample grid visualization:")
min_x = min(coord[0] for coord in grid_example.red_tiles)
min_y = min(coord[1] for coord in grid_example.red_tiles)
max_x = max(coord[0] for coord in grid_example.red_tiles)
max_y = max(coord[1] for coord in grid_example.red_tiles)

for y in range(min_y, max_y + 1):
    line = ""
    for x in range(min_x, max_x + 1):
        if (x, y) in grid_example.red_tiles:
            line += '#'
        elif (x, y) in grid_example.green_tiles:
            line += 'O'
        else:
            line += '.'
    print(line)

Checking rectangles among 8 red tiles...
  Total rectangles checked: 9

Largest rectangle in example:
Area: 15
Corners: (7, 1, 9, 5)

Example grid visualization:
.....#OOO#
.....O...O
#OOOO#...O
O........O
#OOOOOO#.O
.......O.O
.......#O#
